In [ ]:
#import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from pylab import *
import keras
import keras.utils
%matplotlib inline

### load  the data

In [ ]:
df=pd.read_csv(r'C:\Users\ichra\Downloads\fer2013.csv')
df

0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [ ]:
print("Information about data: ") 
df.info()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
# 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
df['emotion'].value_counts().plot(kind='bar')
plt.title('number of emotions')
plt.xlabel('emotions')
plt.ylabel('Count')
sns.despine

In [ ]:
df.Usage

In [ ]:
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
len_angry = len(df["emotion"][df.emotion == 0])
len_Disgust = len(df["emotion"][df.emotion == 1])
len_Fear = len(df["emotion"][df.emotion == 2])
len_Happy = len(df["emotion"][df.emotion == 3])
len_Sad = len(df["emotion"][df.emotion == 4])
len_Surprise = len(df["emotion"][df.emotion == 5])
len_Neutral= len(df["emotion"][df.emotion == 6])

arr = np.array([len_angry,len_Disgust,len_Fear,len_Happy,len_Sad,len_Surprise,len_Neutral]) 
labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral'] 
print("Total of Angry set :- ", len_angry)
print("Total of Disgust set :- ", len_Disgust)
print("Total of fear set :- ", len_Fear)
print("Total of happy set :- ", len_Happy)
print("Total of sad set :- ", len_Sad)
print("Total of surprise set :- ", len_Surprise)
print("Total of neutral set :- ", len_Neutral)


plt.pie(arr, labels=labels, explode = [0.2,0.2,0.2,0.2,0.2,0.2,0.2] , shadow=True) 
plt.show() 

In [ ]:
df["Usage"]

# Data development

In [ ]:
num_classes = 7 #0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
#initialize trainset and test set
x_train, y_train, x_test, y_test = [], [], [], []

for i in range(len(df)):
    emotion=df['emotion'][i]
    usage=df['Usage'][i]
    val = df['pixels'][i].split(" ")
    pixels = np.array(val, 'float32')
    
    emotion = keras.utils.to_categorical(emotion, num_classes)
    
    if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
    elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
  
    

In [ ]:
#data transformation for train and test sets
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')


In [ ]:
x_train /= 255 #normalize inputs between [0, 1] to be in the same scale
x_test /= 255


In [ ]:
print ("the samples train: " ,x_train.shape)
print("the samples test:", x_test.shape)

In [ ]:
# y_train and y_test should be a 1d array
y=pd.DataFrame(y_train)
y = y.apply(lambda x: x.argmax(), axis=1)
y=np.array(y)
y_t=pd.DataFrame(y_test)
y_t=y_t.apply(lambda x: x.argmax(), axis=1)
y_t=np.array(y_t)

# Model Development

First we will extrat features using the Histogram of Oriented Gradient extractor (HOG), Then put these features in the suport vector machine (svm) algorithm and calculate the accuracy  

In [ ]:
#HOG Features 
from skimage.feature import hog

In [ ]:
#reshape the train and test set
x_train = x_train.reshape(x_train.shape[0], 48,48,1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48,48,1)
x_test = x_test.astype('float32')


In [ ]:
#extracting HOG features from train set
X_Hog=[]
for i in range (len(x_train)):
    fd, hog_image = hog(x_train[i], orientations=9, pixels_per_cell=(6, 6),
                	cells_per_block=(2, 2), visualize=True, multichannel=True)
    X_Hog.append(hog_image)
X_Hog

In [ ]:
X_Hog=np.array(X_Hog, 'float32')
nsamples, nx, ny = X_Hog.shape
X2_Hog = X_Hog.reshape((nsamples,nx*ny))


In [ ]:
#extracting HOG features from test set 
Xt_Hog=[]
for i in range (len(x_test)):
    fd, hog_image = hog(x_test[i], orientations=9, pixels_per_cell=(16, 16),
                	cells_per_block=(2, 2), visualize=True, multichannel=True)
    Xt_Hog.append(hog_image)


In [ ]:
Xt_Hog=np.array(Xt_Hog, 'float32')
nsamples, nx, ny = Xt_Hog.shape
Xt2_Hog = Xt_Hog.reshape((nsamples,nx*ny))

# SVM classifier

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV 

grid = GridSearchCV(LinearSVC(dual=False), {'C': [1.0, 0.01,0.001,10.0,2.0,4.0]},cv=3)
grid.fit(X2_Hog, y)

In [ ]:
# make predictions on test set
pred=grid.predict(Xt2_Hog)

In [ ]:
# compute and print accuracy score
# import metrics to compute accuracy
from sklearn.metrics import accuracy_score
print('Model accuracy score:{0:0.4f}'. format(accuracy_score(y_t, pred)))

# Implement Deep Learning model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator 
datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range = 10,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode = 'nearest')

In [ ]:
testgen = ImageDataGenerator(rescale=1./255)
datagen.fit(x_train)

In [ ]:
batch_size = 12

In [ ]:
train_flow = datagen.flow(x_train, y_train, batch_size=batch_size) 
test_flow = testgen.flow(x_test, y_test, batch_size=batch_size)

In [ ]:
#import the necessary moduls
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization,Concatenate
from keras.layers.convolutional import Conv2D,Conv1D
from keras.layers.pooling import MaxPooling2D, MaxPooling1D
from keras.layers import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping

In [ ]:
def facial_emotion(input_shape=(48,48,1)):
    # first input model
    visible = Input(shape=input_shape, name='input')
    num_classes = 7
    #the 1-st block
    conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
    conv1_1 = BatchNormalization()(conv1_1)
    conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
    drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)
    #the 2-nd block
    conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
    conv2_1 = BatchNormalization()(conv2_1)
    conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
    conv2_2 = BatchNormalization()(conv2_2)
    conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
    conv2_2 = BatchNormalization()(conv2_3)
    pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
    drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)
    print(model.summary())
    
    return model

In [ ]:
#Compliling the model with adam optimixer and categorical crossentropy loss
model_cnn =facial_emotion()
opt = Adam(lr=0.0001, decay=1e-6)
model_cnn.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
num_epochs =20 
history = model_cnn.fit_generator(train_flow, 
                    steps_per_epoch=len(x_train) / batch_size, 
                    epochs=num_epochs,  
                    verbose=1,  
                    validation_data=test_flow)

In [ ]:
train_score = model_cnn.evaluate(x_train, y_train, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
 
test_score = model_cnn.evaluate(,x_test, y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

# HOG+CNN

In [ ]:
#merging HOG features and cnn features
model_1D_input = Input(shape=(48,48))
#model_1D=Dense(2304, activation='relu',kernel_regularizer=l2(0.001))(model_1D_input)
model_1D = Conv1D(32, kernel_size=3 , strides=1, activation='relu',kernel_regularizer=l2(0.0001))(model_1D_input)
model_1D = Conv1D(64, kernel_size=3 , strides=1, activation='relu',kernel_regularizer=l2(0.0001))(model_1D)
model_1D = MaxPooling1D(pool_size=2, strides= 2)(model_1D)
model_1D=Dropout(0.25)(model_1D)
model_1D = Conv1D(128, kernel_size=3 , strides=1, activation='relu',kernel_regularizer=l2(0.0001))(model_1D)
model_1D = Conv1D(256, kernel_size=3 , strides=1, activation='relu',kernel_regularizer=l2(0.0001))(model_1D)
model_1D = MaxPooling1D(pool_size=2, strides=2)(model_1D)
model_1D=Dropout(0.3)(model_1D)
model_1D = Flatten()(model_1D)
model_1D = Dense(2304 , activation='relu',kernel_regularizer=l2(0.0001))(model_1D)



model_2D_input = Input((48,48,1))
model_2D = Conv2D(32, kernel_size=(3, 3) , strides=(1,1), activation='relu',kernel_regularizer=l2(0.0001))(model_2D_input)
model_2D = Conv2D(64, kernel_size=(3, 3) , strides=(1,1), activation='relu',kernel_regularizer=l2(0.0001))(model_2D)
model_2D = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(model_2D)
model_2D=Dropout(0.25)(model_2D)
model_2D = Conv2D(128, kernel_size=(3, 3) , strides=(1,1), activation='relu',kernel_regularizer=l2(0.0001))(model_2D)
model_2D = Conv2D(256, kernel_size=(3, 3) , strides=(1,1), activation='relu',kernel_regularizer=l2(0.0001))(model_2D)
model_2D = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(model_2D)
model_2D=Dropout(0.3)(model_2D)
model_2D = Flatten()(model_2D)
model_2D = Dense(2304 , activation='relu',kernel_regularizer=l2(0.0001))(model_2D)



merged = Concatenate()([model_1D,model_2D])
output = Dense(7, activation='softmax')(merged)

model_HOG_cnn = Model(inputs=[model_1D_input,model_2D_input], outputs=[output])


In [ ]:
model_HOG_cnn.summary()

In [ ]:
model_HOG_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model_HOG_cnn.fit([X_Hog,x_train],y_train,epochs=100,  
                    verbose=1,  
                    validation_data=([Xt_Hog,x_test],y_test),callbacks = [EarlyStopping(monitor='val_accuracy', patience=5)])

In [ ]:
train_score = model_HOG_cnn.evaluate([X_Hog,x_train], y_train, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
 
test_score = model_HOG_cnn.evaluate([Xt_Hog,x_test], y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

# save the best model 

In [ ]:
model_json = model_HOG_cnn.to_json()
with open("model_final.json", "w") as json_file:
    json_file.write(model_json)
model_HOG_cnn.save_weights("model_final.h5")
print("Saved model to disk")

In [ ]:
from tensorflow.keras.models import model_from_json
model = model_from_json(open("model_final.json", "r").read())
model.load_weights('model_final.h5')

In [ ]:
import cv2
face_haar_cascade = cv2.CascadeClassifier(r'C:\Users\ichra\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')

In [ ]:
faceCascade = cv2.CascadeClassifier(r'C:\Users\ichra\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)
model = model_from_json(open("model_final.json", "r").read())
model.load_weights('model_final.h5')
model.get_config()

target = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
font = cv2.FONT_HERSHEY_SIMPLEX
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1)

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2, 5)
        face_crop = frame[y:y + h, x:x + w]
        face_crop = cv2.resize(face_crop, (48, 48))
        f=face_crop
        face_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
        face_crop = np.asarray(face_crop)
        #face_crop = np.expand_dims(face_crop, axis = 2)
        face_crop =face_crop.reshape(-1,48,48,1)
        fd, hog_image = hog(f, orientations=9, pixels_per_cell=(2, 2),
                	cells_per_block=(2, 2), visualize=True, multichannel=True)
        hog_image = (255-hog_image) / 255                # normalize as white on black
        hog_image=np.expand_dims(hog_image, axis=0) # will move it to (1,28,28)
        result = target[np.argmax(model.predict([hog_image,face_crop]))]
        cv2.putText(frame, result, (x, y), font, 1, (200, 0, 0), 3, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [ ]:
video_capture.release()
